#### Load the Dataset with Image Paths
##### Update the dataset path and use ImageFolder to load the images

In [7]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the dataset path
dataset_path = "C:\ICT Project 2024\Image"

# Define image transformations (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to match CLIP preprocessing
])

# Load dataset using ImageFolder
train_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Create DataLoader
batch_size = 22  # Adjust as needed
dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Print class names to verify
print(f"Classes: {train_dataset.classes}")

Classes: ['3D Design', 'AI-abstract Imagery', 'Asymmetry', 'Augmented Reality', 'Brutalism', 'Card-based Design', 'City', 'Dark Mode', 'Flat Design', 'Glass Morphism', 'Gradiant Design', 'Illustrative Design', 'Interactive Gradients', 'Material Design', 'Memphis Design', 'MetroUI', 'Minimalism', 'Neomorphism', 'Parallax Scrolling', 'Responsive Design', 'RetroVintage Design', 'Skeuomorphism', 'Typography-focused']


<>:5: SyntaxWarning: invalid escape sequence '\I'
<>:5: SyntaxWarning: invalid escape sequence '\I'
C:\Users\dammi\AppData\Local\Temp\ipykernel_20680\3050682020.py:5: SyntaxWarning: invalid escape sequence '\I'
  dataset_path = "C:\ICT Project 2024\Image"


#### Load CLIP and the tokenizer / Import Required Libraries / Update Classifier with Number of Classes

In [8]:
import clip
import torch
from torch import nn, optim
import clip
import torch
from PIL import Image

# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Update the classifier to match the number of design styles
num_classes = len(train_dataset.classes)  # Automatically counts subfolders
classifier = nn.Linear(model.visual.output_dim, num_classes).to(device)

#### Fix (Define the Loss Function (criterion))

In [9]:
criterion = nn.CrossEntropyLoss()

#### Update the Training Loop

In [10]:
from torch import nn, optim  # Import necessary modules

# Freeze CLIP encoder
for param in model.parameters():
    param.requires_grad = False

# Add a custom classification layer (already defined earlier)
# classifier = nn.Linear(model.visual.output_dim, num_classes).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Define the loss function
optimizer = optim.Adam(classifier.parameters(), lr=1e-4)  # Define optimizer

# Define the training loop
epochs = 50  # Number of training epochs
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in dataloader:  # Use DataLoader for batches
        # Move data to the device (CPU/GPU)
        images, labels = images.to(device), labels.to(device)

        # Extract features using the frozen CLIP encoder
        with torch.no_grad():
            features = model.encode_image(images)
        
        # Forward pass through the classifier
        predictions = classifier(features)
        loss = criterion(predictions, labels)  # Compute the loss
        
        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Accumulate loss
        running_loss += loss.item()
    
    # Print loss for each epoch
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(dataloader):.4f}")

Epoch 1/50, Loss: 3.2375
Epoch 2/50, Loss: 3.2199
Epoch 3/50, Loss: 3.1955
Epoch 4/50, Loss: 3.1847
Epoch 5/50, Loss: 3.1638
Epoch 6/50, Loss: 3.1430
Epoch 7/50, Loss: 3.1261
Epoch 8/50, Loss: 3.1088
Epoch 9/50, Loss: 3.0914
Epoch 10/50, Loss: 3.0681
Epoch 11/50, Loss: 3.0511
Epoch 12/50, Loss: 3.0321
Epoch 13/50, Loss: 3.0213
Epoch 14/50, Loss: 2.9922
Epoch 15/50, Loss: 2.9823
Epoch 16/50, Loss: 2.9607
Epoch 17/50, Loss: 2.9492
Epoch 18/50, Loss: 2.9228
Epoch 19/50, Loss: 2.9075
Epoch 20/50, Loss: 2.8816
Epoch 21/50, Loss: 2.8741
Epoch 22/50, Loss: 2.8573
Epoch 23/50, Loss: 2.8413
Epoch 24/50, Loss: 2.8198
Epoch 25/50, Loss: 2.8058
Epoch 26/50, Loss: 2.7943
Epoch 27/50, Loss: 2.7626
Epoch 28/50, Loss: 2.7607
Epoch 29/50, Loss: 2.7350
Epoch 30/50, Loss: 2.7249
Epoch 31/50, Loss: 2.7066
Epoch 32/50, Loss: 2.7002
Epoch 33/50, Loss: 2.6724
Epoch 34/50, Loss: 2.6508
Epoch 35/50, Loss: 2.6375
Epoch 36/50, Loss: 2.6177
Epoch 37/50, Loss: 2.6031
Epoch 38/50, Loss: 2.5963
Epoch 39/50, Loss: 2.

#### Save the Model & Re-load

In [11]:
torch.save(classifier.state_dict(), "style_classifier.pth")

In [12]:
classifier.load_state_dict(torch.load("style_classifier.pth"))
classifier.eval()

C:\Users\dammi\AppData\Local\Temp\ipykernel_20680\3350549794.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  classifier.load_state_dict(torch.load("style_classifier.pth"

Linear(in_features=512, out_features=23, bias=True)

#### Prepare Design Styles as Text Prompts

In [13]:
# Define design style prompts
design_styles = [
    "Flat Design",
    "Metro UI",
    "Skeuomorphism",
    "Material Design",
    "Neomorphism",
    "Brutalism",
    "Responsive Design",
    "Minimalism",
    "Dark Mode",
    "Glass Morphism",
    "Gradiant Design",
    "3D Design",
    "RetroVintage Design",
    "Asymmetry",
    "Card-based Design",
    "Parallax Scrolling",
    "Typography-focused",
    "Augmented Reality",
    "Illustrative Design",
    "Interactive Gradients",
    "AI-abstract Imagery",
    "Memphis Design",
]

# Tokenize prompts
text_inputs = clip.tokenize(design_styles).to(device)

##### Load and Preprocess the Image,  Calculate Similarities , Visualize Results

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load and preprocess image
image_path = r"C:\ICT Project 2024\Images\City\Turku.jpg"  # Path to your website screenshot
image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)

# Generate image and text embeddings
with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text_inputs)

# Normalize features
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)

# Calculate cosine similarity
similarity = (image_features @ text_features.T).squeeze(0)

# Get top match
top_idx = similarity.argmax().item()
print(f"The website design style is most likely: {design_styles[top_idx]}")

# Optionally, print all scores
scores = {design_styles[i]: similarity[i].item() for i in range(len(design_styles))}
print("All similarity scores:", scores)

# Visualization
plt.figure(figsize=(8, 8))
plt.imshow(Image.open(image_path))  # Display the image
plt.axis('off')  # Remove axes for better display
plt.title(f"Predicted Style: {design_styles[top_idx]}\nScore: {similarity[top_idx].item():.4f}", fontsize=14)
plt.show()

NameError: name 'preprocess' is not defined